In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-geometric

In [ ]:
def setup_seed(seed, cuda):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if cuda is True:
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
setup_seed(123, torch.cuda.is_available())
random.seed(123)

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa

dataset = 'Pubmed'
path = osp.join('', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, dim, cached=True,
                             normalize= 1)
        self.conv0 = GCNConv(dim, dataset.num_classes, cached=True,
                             normalize= 1)
        self.keep = nn.Softmax()

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = self.inflation(self.keep(self.conv1(x, edge_index, edge_weight)), config['power'])
        x = F.dropout(x, training=self.training)

        x = self.conv0(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

    def inflation(self, x, pow):

        power = pow
        x = x / x.sum(-1).unsqueeze(-1).expand_as(x)
        x = x.pow(power)
        x = x / x.sum(-1).unsqueeze(-1).expand_as(x)

        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv0.parameters(), weight_decay=0)
], lr=0.05)  # Only perform weight-decay on first convolution.

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    a = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
    return a

@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 1501):
    a = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    if epoch % 5 == 0:
      log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Loss: {:.4f}'
      print(log.format(epoch, train_acc, best_val_acc, test_acc, a))

In [ ]:
config = {
    'seed': 123456,
    'power': 2
}